<a href="https://colab.research.google.com/github/alierenc/di725-transformers-and-attention-based-deep-networks-term-project/blob/main/Phase%20III/2.1.%20SigLIP-GPT2%20Custom%20VLM%20-%20Image%20Captioning%20Fine-tuning%20and%20Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import login
hf_token = " " # Huggingface token
login(token = hf_token)

In [ ]:
# Access google drive to save the model
from google.colab import drive
drive.mount('/content/drive')

# Import and log in wandb
import wandb

wandb.login()
# Initialize W&B run
wandb.init(project="term-project-vision-language-model", name="siglip-gpt2")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


wandb: Currently logged in as: aeren to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
!pip install -U datasets
!pip install bitsandbytes --upgrade

In [ ]:
from datasets import load_dataset, DatasetDict

# Load the dataset of full riscm
ds = load_dataset('caglarmert/full_riscm')

full = ds["train"]

# test   = indices [0, 3150)
test_ds = full.select(range(3150))

# validation = indices [3150, 6300)
val_ds = full.select(range(3150, 6300))

# train  = indices [6300, end)
train_ds = full.select(range(6300, len(full)))

# bundle into a DatasetDict
ds = DatasetDict({
    "test": test_ds,
    "val": val_ds,
    "train": train_ds,
})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
ds["test"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
 'caption_1': 'A gray plane on the runway and the lawn beside .',
 'caption_2': 'A grey plane is on the runway by the lawn .',
 'caption_3': 'There is an airplane on the runway with a large lawn by the runway .',
 'caption_4': 'A plane is parked on the runway next to the grass .',
 'caption_5': 'There is a plane on the runway beside the grass .'}

In [ ]:
import torch
import torch.nn as nn

class CustomVLM(nn.Module):
    def __init__(self, vision_model, language_model, vision_hidden_size, language_hidden_size):
        super(CustomVLM, self).__init__()
        self.vision_model = vision_model
        self.language_model = language_model
        self.vision_proj = nn.Linear(vision_hidden_size, language_hidden_size)

    def forward(self, image, input_ids=None, attention_mask=None, labels=None):
        # Encode image
        vision_output = self.vision_model(pixel_values=image).last_hidden_state
        vision_embedding = torch.mean(vision_output, dim=1)
        projected_embedding = self.vision_proj(vision_embedding)  # [B, D]
        prefix = projected_embedding.unsqueeze(1)  # [B, 1, D]

        # Get text embeddings
        inputs_embeds = self.language_model.transformer.wte(input_ids)  # [B, T, D]
        inputs_embeds = torch.cat([prefix, inputs_embeds], dim=1)

        # Create or update attention mask
        batch_size = inputs_embeds.size(0)
        seq_len = inputs_embeds.size(1)

        if attention_mask is None:
            # No mask was passed -> assume no padding in input
            attention_mask = torch.ones((batch_size, input_ids.size(1)), dtype=torch.long, device=input_ids.device)

        # Prefix mask for vision token
        prefix_mask = torch.ones((batch_size, 1), dtype=attention_mask.dtype, device=attention_mask.device)
        attention_mask = torch.cat([prefix_mask, attention_mask], dim=1)

        return self.language_model(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            labels=labels
        )


In [ ]:
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    SiglipVisionModel,
    AutoImageProcessor,
    BitsAndBytesConfig
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import bitsandbytes

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load quantized GPT-2
language_model = GPT2LMHeadModel.from_pretrained(
    "gpt2",
    quantization_config=bnb_config,
    device_map="auto"
)

# Prepare for QLoRA
language_model = prepare_model_for_kbit_training(language_model)

# LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
language_model = get_peft_model(language_model, lora_config)
language_hidden_size = language_model.config.n_embd  # Usually 768

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Load SigLIP
vision_model = SiglipVisionModel.from_pretrained("google/siglip-base-patch16-224")
vision_model.requires_grad_(False)
image_processor = AutoImageProcessor.from_pretrained("google/siglip-base-patch16-224")
vision_hidden_size = vision_model.config.hidden_size

# Final model
model = CustomVLM(
    vision_model=vision_model,
    language_model=language_model,
    vision_hidden_size=vision_hidden_size,
    language_hidden_size=language_hidden_size
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
print("Language model device:", next(model.language_model.parameters()).device)
print("Vision model device:", next(model.vision_model.parameters()).device)
print("Vision projection layer device:", next(model.vision_proj.parameters()).device)

Language model device: cuda:0
Vision model device: cuda:0
Vision projection layer device: cuda:0


In [ ]:
# We count the number of parameters
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {"Total": total, "Trainable": trainable}

print("Printing the total number of parameters and the number of trainable parameters:")
print("Vision Encoder (SigLIP):", count_parameters(model.vision_model))
print("Language Decoder (GPT-2):", count_parameters(model.language_model))
print("Vision Projection Layer:", count_parameters(model.vision_proj))
print("Total CustomVLM:", count_parameters(model))

Printing the total number of parameters and the number of trainable parameters:
Vision Encoder (SigLIP): {'Total': 92884224, 'Trainable': 0}
Language Decoder (GPT-2): {'Total': 82783488, 'Trainable': 811008}
Vision Projection Layer: {'Total': 590592, 'Trainable': 590592}
Total CustomVLM: {'Total': 176258304, 'Trainable': 1401600}


In [ ]:
# Check whether the model can generate captions at all
# Set model to eval mode
model.eval()

for i in range(10):
    print(f"Generating caption for sample {i + 1}")

    # Load and preprocess image
    image = ds["test"][i]["image"]
    pixel_values = image_processor(image, return_tensors="pt")["pixel_values"]
    pixel_values = pixel_values.to(model.language_model.device)

    with torch.no_grad():
        # Vision encoding
        vision_output = model.vision_model(pixel_values=pixel_values).last_hidden_state
        vision_embedding = vision_output.mean(dim=1)
        vision_proj = model.vision_proj(vision_embedding).unsqueeze(1)  # [B, 1, D]

        # Prepare prompt
        prompt = "caption en"
        tokenized = tokenizer(prompt, return_tensors="pt", padding=False)
        input_ids = tokenized["input_ids"].to(model.language_model.device)
        attention_mask = tokenized["attention_mask"].to(model.language_model.device)

        # Text embedding
        input_embeds = model.language_model.transformer.wte(input_ids)
        input_embeds = torch.cat([vision_proj, input_embeds], dim=1)

        # Extend attention mask with prefix mask
        prefix_mask = torch.ones((attention_mask.shape[0], 1), dtype=attention_mask.dtype).to(attention_mask.device)
        attention_mask = torch.cat([prefix_mask, attention_mask], dim=1)

        # Generate text
        generated_ids = model.language_model.generate(
            inputs_embeds=input_embeds,
            attention_mask=attention_mask,
            max_new_tokens=30,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False
        )

        # Decode the caption
        caption = tokenizer.decode(generated_ids[0], skip_special_tokens=True)[len(prompt):]
        print("Generated caption:", repr(caption))
        print()


Generating caption for sample 1
Generated caption: 'T-S-S-S-S-S-S-S-S-S-S-S'

Generating caption for sample 2
Generated caption: 'T-S-S-S-S-S-S-S-S-S-S-S'

Generating caption for sample 3
Generated caption: 'm a big, big, big, big, big, big, big, big, big, big, big,'

Generating caption for sample 4
Generated caption: 'I-I-I-I-I-I-I-I-I-I-I-I'

Generating caption for sample 5
Generated caption: 'rst thing that I do is I go to the local, the local, the local, the local, the local, the'

Generating caption for sample 6
Generated caption: 'st of the two, the "The Great and the Great" is a "The Great and the Great" and the "The'

Generating caption for sample 7
Generated caption: 'he other.\n\nThe first time I saw the video, I was in the middle of the night, and I was'

Generating caption for sample 8
Generated caption: 'T-S-S-S-S-S-S-S-S-S-S-S'

Generating caption for sample 9
Generated caption: 'st of the two, the "The Great and the Great" is a "The Great and the Great" and the "The'

Ge

In [ ]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm

# Prefix for all captions
caption_prefix = "caption en"

# Custom collate function
def collate_fn(batch):
    images = [image_processor(example["image"], return_tensors="pt")["pixel_values"].squeeze(0) for example in batch]
    captions = [caption_prefix + example["caption_3"] for example in batch]
    pixel_values = torch.stack(images)

    tokenized = tokenizer(
        captions,
        padding=True,
        truncation=True,
        return_tensors="pt",
        return_attention_mask=True
    )

    return {
        "pixel_values": pixel_values,
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": tokenized["input_ids"].clone()
    }

# DataLoaders
train_loader = DataLoader(ds["train"], batch_size=256, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(ds["val"], batch_size=256, shuffle=False, collate_fn=collate_fn)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_samples = 0

    for step, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}")):
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Pad labels for prefix position with -100
        prefix_ignore = torch.full((labels.size(0), 1), -100, dtype=labels.dtype, device=labels.device)
        labels = torch.cat([prefix_ignore, labels], dim=1)

        # Add prefix to attention mask
        prefix_mask = torch.ones((attention_mask.size(0), 1), dtype=attention_mask.dtype, device=attention_mask.device)
        extended_attention_mask = torch.cat([prefix_mask, attention_mask], dim=1)

        optimizer.zero_grad()

        outputs = model(
            image=pixel_values,
            input_ids=input_ids,
            attention_mask=extended_attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        batch_size = input_ids.size(0)
        total_loss += loss.item() * batch_size
        total_samples += batch_size

        wandb.log({
            "train/loss": loss.item(),
            "train/step": epoch * len(train_loader) + step
        })

    avg_train_loss = total_loss / total_samples
    print(f"Epoch {epoch+1} completed. Average Train Loss: {avg_train_loss:.4f}")
    wandb.log({"train/avg_epoch_loss": avg_train_loss, "epoch": epoch + 1})

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_samples = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Pad labels for prefix position with -100
            prefix_ignore = torch.full((labels.size(0), 1), -100, dtype=labels.dtype, device=labels.device)
            labels = torch.cat([prefix_ignore, labels], dim=1)

            prefix_mask = torch.ones((attention_mask.size(0), 1), dtype=attention_mask.dtype, device=attention_mask.device)
            extended_attention_mask = torch.cat([prefix_mask, attention_mask], dim=1)

            outputs = model(
                image=pixel_values,
                input_ids=input_ids,
                attention_mask=extended_attention_mask,
                labels=labels
            )

            batch_size = input_ids.size(0)
            val_loss += outputs.loss.item() * batch_size
            val_samples += batch_size

    avg_val_loss = val_loss / val_samples
    print(f"Average Validation Loss: {avg_val_loss:.4f}")
    wandb.log({"val/loss": avg_val_loss, "epoch": epoch + 1})

Epoch 1/10:   0%|          | 0/150 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Epoch 1/10: 100%|██████████| 150/150 [04:42<00:00,  1.89s/it]


Epoch 1 completed. Average Train Loss: 3.2993


Validating: 100%|██████████| 13/13 [00:17<00:00,  1.37s/it]


Average Validation Loss: 2.3526


Epoch 2/10: 100%|██████████| 150/150 [04:42<00:00,  1.88s/it]


...

Average Validation Loss: 0.6536


Epoch 10/10: 100%|██████████| 150/150 [04:41<00:00,  1.88s/it]


Epoch 10 completed. Average Train Loss: 0.6684


Validating: 100%|██████████| 13/13 [00:17<00:00,  1.37s/it]

Average Validation Loss: 0.6095


In [ ]:
# Set model to evaluation mode
model.eval()
predictions = []

# Loop through the full test dataset
for i in tqdm(range(len(ds["test"])), desc="Generating captions"):
    image = ds["test"][i]["image"]
    pixel_values = image_processor(image, return_tensors="pt")["pixel_values"].to(model.language_model.device)

    with torch.no_grad():
        # Vision encoding
        vision_output = model.vision_model(pixel_values=pixel_values).last_hidden_state
        vision_embedding = vision_output.mean(dim=1)
        vision_proj = model.vision_proj(vision_embedding).unsqueeze(1)  # [B, 1, D]

        # Prepare prompt
        prompt = "caption en"
        tokenized = tokenizer(prompt, return_tensors="pt", padding=False)
        input_ids = tokenized["input_ids"].to(model.language_model.device)
        attention_mask = tokenized["attention_mask"].to(model.language_model.device)

        # Text embedding
        input_embeds = model.language_model.transformer.wte(input_ids)
        input_embeds = torch.cat([vision_proj, input_embeds], dim=1)

        # Extend attention mask with prefix mask
        prefix_mask = torch.ones((attention_mask.shape[0], 1), dtype=attention_mask.dtype).to(attention_mask.device)
        attention_mask = torch.cat([prefix_mask, attention_mask], dim=1)

        # Generate text
        generated_ids = model.language_model.generate(
            inputs_embeds=input_embeds,
            attention_mask=attention_mask,
            max_new_tokens=30,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False
        )

        # Decode caption (removing prompt part)
        caption = tokenizer.decode(generated_ids[0], skip_special_tokens=True)[len(prompt):].strip()
        predictions.append(caption)

Generating captions: 100%|██████████| 3150/3150 [16:12<00:00,  3.24it/s]


In [ ]:
# Get the references
# Define a variable to store the reference captions
all_references = []
for i in tqdm(range(len(ds["test"])), desc="Collecting reference captions"):
    # Get the reference
    reference_per_sample = []
    for j in range(1,6):
        reference = ds["test"][i][f"caption_{j}"]
        reference_per_sample.append(reference)
        print(f"The reference caption_{j}:")
        print(repr(reference))

    print()
    all_references.append(reference_per_sample)

The reference caption_1:
'A gray plane on the runway and the lawn beside .'
The reference caption_2:
'A grey plane is on the runway by the lawn .'
The reference caption_3:
'There is an airplane on the runway with a large lawn by the runway .'
The reference caption_4:
'A plane is parked on the runway next to the grass .'
The reference caption_5:
'There is a plane on the runway beside the grass .'

The reference caption_1:
'Three small planes parked in a line on the airport and a big plane behind them .'
The reference caption_2:
'There are four aircraft on the open ground, The largest of which is three times as large as the smallest one .'
The reference caption_3:
'There are many planes of different sizes in a clearing .'
The reference caption_4:
'Four planes are parked on the runway .'
The reference caption_5:
'Four planes of different sizes were on the marked ground .'

The reference caption_1:
'A plane parked in a line on the airport with some marks .'
The reference caption_2:
'A whit

The reference caption_1:
'Two tail-to-tail planes parked on the airport .'
The reference caption_2:
'Here are two planes heading in opposite directions .'
The reference caption_3:
'There are two planes of different sizes on a clearing .'
The reference caption_4:
'Two planes are parked on the open space between the terminal and the runway .'
The reference caption_5:
'Two planes are on the marked ground .'

The reference caption_1:
'Two planes with the purple marks parked next to boarding bridges .'
The reference caption_2:
'Two blue-tailed planes parked at the boarding gate .'
The reference caption_3:
'Two planes of the same size parked neatly next to the buildings inside the airport .'
The reference caption_4:
'Two planes are parked next to the terminal .'
The reference caption_5:
'Two planes are in the parking lot next to the building .'

The reference caption_1:
'Two planes parked one after another on the airport and the lawn beside .'
The reference caption_2:
'There are two white pl

The reference caption_1:
'An airport with some parallel runways and an open area on the lawn .'
The reference caption_2:
'The lawn is divided into many rectangular areas by runways .'
The reference caption_3:
'A huge complex airport with many runways and buildings and many planes parked next to the brown mountains .'
The reference caption_4:
'There is an airport between the mountain and many houses .'
The reference caption_5:
'The two roads of the airport are parallel to each other .'

The reference caption_1:
'An airport with two parallel runways and an open area at the foot of the mountain .'
The reference caption_2:
'This airport has only one runway .'
The reference caption_3:
'There are many buildings along the runway and apron of the airport and there are many planes parked  On the other side of the runway is a brown mountain range .'
The reference caption_4:
'Airports built at the foot of the mountain next to the tenantless land .'
The reference caption_5:
'The two roads of the a

The reference caption_5:
'The two roads of the airport are parallel to each other .'

The reference caption_1:
'An airport with two parallel runways and an open area on the farmland and some buildings beside .'
The reference caption_2:
'The airport is built by the sea and the sea water is light blue .'
The reference caption_3:
'A complex airport with many runways and buildings is surrounded by a brown and green farmland .'
The reference caption_4:
'An airport is surrounded by the field next to many houses .'
The reference caption_5:
'There are many buildings of different sizes and shapes at the airport .'

The reference caption_1:
'An airport with a runway and an open area on the farmland with different plants and some villages beside .'
The reference caption_2:
'This airport has only one runway, And its color is grey .'
The reference caption_3:
'An airport with a runway and many buildings is surrounded by a brown and green farmland .'
The reference caption_4:
'There is an airport betw

'A road is next to a baseball diamond .'

The reference caption_1:
'A baseball diamond next to a parking lot and the medium residential area beside .'
The reference caption_2:
'There are 11 baseball courts in the picture, Four of which are double the others .'
The reference caption_3:
'There is a baseball diamond in the grass surrounded by many trees, Houses and parking lot .'
The reference caption_4:
'A baseball diamond is in the middle of grass surrounded by many buildings and trees .'
The reference caption_5:
'Many buildings are next to  the  baseball diamond .'

The reference caption_1:
'Two baseball diamonds next to a parking lot and some trees beside .'
The reference caption_2:
'In the picture there is a baseball field and a parking lot .'
The reference caption_3:
'There are two baseball diamonds in the grass surrounded by many trees, Houses and parking lot .'
The reference caption_4:
'A big baseball diamond and a small baseball diamond are next to some cars .'
The reference capt

...



The reference caption_1:
'Many trees are planted in the terraces which are in the middle of the image .'
The reference caption_2:
'A large layer of brown and green mixed terraces .'
The reference caption_3:
'The bare terrace is next to some trees .'
The reference caption_4:
'The green plants are on the terrace .'
The reference caption_5:
'There are many yellow terraces of different sizes .'

The reference caption_1:
'Half of the terraces are full of crops, and the other half are empty .'
The reference caption_2:
'A large layer of brown and green mixed terraces with several winding narrow paths .'
The reference caption_3:
'The bare and green terrace is next to some trees .'
The reference caption_4:
'Many green plants are on the terrace .'
The reference caption_5:
'There are green meadows beside the terraces .'

The reference caption_1:
'There is a path between the terraces leading to a building .'
The reference caption_2:
'There are paths and houses in a patch of brown and green terra

The reference caption_1:
'There are two thermal power stations at the top right of the picture, and the left one is emitting smoke .'
The reference caption_2:
'In the thermal power plant there are plant buildings and machinery, two chimneys are emitting smoke .'
The reference caption_3:
'The thermal power station is next to some buildings with white mist above .'
The reference caption_4:
'There are many buildings of different shapes next to the thermal power station .'
The reference caption_5:
'There are some buildings and three chimneys on the thermal power station .'

The reference caption_1:
'Here is a smoking thermal power station with a row of red houses next to it .'
The reference caption_2:
'There are plant and machinery in the thermal power station, and two cooling towers emit steam .'
The reference caption_3:
'The thermal power station is next to some buildings and trees with white mist above .'
The reference caption_4:
'Many buildings of different shapes are next to the therm

The reference caption_1:
'There are seven thermal power stations here, two of which are thinner and taller than others .'
The reference caption_2:
'In the thermal power station there are neatly planned plants and equipment, there is a cooling tower with steam, and a chimney .'
The reference caption_3:
'The thermal power station is on the grass next to some buildings with white mist above .'
The reference caption_4:
'There are many buildings next to the thermal power station .'
The reference caption_5:
'There are many buildings and a chimney on the thermal power station beside a green meadow .'

The reference caption_1:
'Here are three thermal power stations, two of which are emitting smoke .'
The reference caption_2:
'There is a cooling tower in a thermal power station .'
The reference caption_3:
'The thermal power station is on the grass next to some trees .'
The reference caption_4:
'Numerous buildings of different shapes are next to the thermal power station .'
The reference caption

The reference caption_3:
'The wetland consists of dry and green plants and water .'
The reference caption_4:
'There are many green plants in the wetland .'
The reference caption_5:
'There are many buildings near the wetland .'

The reference caption_1:
'Except for the waters, this wetland is covered by vegetation .'
The reference caption_2:
'There is a large land mixed with green and brown in the river in a wetland .'
The reference caption_3:
'The wetland consists of bare land, Green plants and water .'
The reference caption_4:
'There are many different shapes and sizes of water around the wetlands .'
The reference caption_5:
'There are much bare land and many water areas on the wetland .'

The reference caption_1:
'There are several scattered trees in the middle of this wetland .'
The reference caption_2:
'There is a large green area in a lake in a wetland with lush green plants .'
The reference caption_3:
'The wetland consists of green plants and water .'
The reference caption_4:
'Th

The reference caption_4:
'Many different shapes of wetlands are surrounded by water .'
The reference caption_5:
'There are many green islands on the wetland .'

The reference caption_1:
'Except for the waters, this wetland is covered by vegetation .'
The reference caption_2:
'There is a large area of \u200b\u200bdense brown bare land in a lake in a wetland .'
The reference caption_3:
'The wetland consists of dry and green plants and water .'
The reference caption_4:
'Many waters of different sizes are surrounded by wetlands .'
The reference caption_5:
'There are much bare land and many water areas on the wetland .'

The reference caption_1:
'There are several small islands in the waters of this wetland .'
The reference caption_2:
'There are a few brown bare lands in the river in a wetland with sparse green plants on the bare lands .'
The reference caption_3:
'The wetland consists of dry and green plants and water .'
The reference caption_4:
'The green plants on the left of the wetland 

In [ ]:
# Check the format of the reference captions
print(all_references[:5])

[['A gray plane on the runway and the lawn beside .', 'A grey plane is on the runway by the lawn .', 'There is an airplane on the runway with a large lawn by the runway .', 'A plane is parked on the runway next to the grass .', 'There is a plane on the runway beside the grass .'], ['Three small planes parked in a line on the airport and a big plane behind them .', 'There are four aircraft on the open ground, The largest of which is three times as large as the smallest one .', 'There are many planes of different sizes in a clearing .', 'Four planes are parked on the runway .', 'Four planes of different sizes were on the marked ground .'], ['A plane parked in a line on the airport with some marks .', 'A white plane was parked on the instruction line .', 'An airplane parked in an open area with many containers next to it .', 'A plane is parked on the open space .', 'There is 1 plane on the ground marked .'], ['A small plane and a big plane parked next to boarding bridges .', 'A white plan

In [ ]:
# Check the format of the predicted captions. Each sample starts with a new line
print(predictions[:5])

['is on the grass next to some trees .', 'is on the grass next to some buildings .', 'is on the grass next to some buildings .', 'many planes on the runway at the airport, There are many cars on the runway, And there are many buildings on the runway .', 'is on the grass next to some trees .']


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction

nltk.download('punkt_tab')
nltk.download('punkt')

# Tokenize references and predictions:
tokenized_refs = [
    [nltk.word_tokenize(ref.lower()) for ref in refs]
    for refs in all_references
]

tokenized_hyps = [nltk.word_tokenize(pred.lower()) for pred in predictions]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tokenized_refs[0]

[['a',
  'gray',
  'plane',
  'on',
  'the',
  'runway',
  'and',
  'the',
  'lawn',
  'beside',
  '.'],
 ['a', 'grey', 'plane', 'is', 'on', 'the', 'runway', 'by', 'the', 'lawn', '.'],
 ['there',
  'is',
  'an',
  'airplane',
  'on',
  'the',
  'runway',
  'with',
  'a',
  'large',
  'lawn',
  'by',
  'the',
  'runway',
  '.'],
 ['a',
  'plane',
  'is',
  'parked',
  'on',
  'the',
  'runway',
  'next',
  'to',
  'the',
  'grass',
  '.'],
 ['there',
  'is',
  'a',
  'plane',
  'on',
  'the',
  'runway',
  'beside',
  'the',
  'grass',
  '.']]

In [ ]:
# Sentence-level BLEU-2
smooth = SmoothingFunction().method1
for i, (refs_per_sample, hyp_tok) in enumerate(zip(tokenized_refs, tokenized_hyps)):
    scores = []
    for refs_tok in refs_per_sample:
        score = sentence_bleu(
            [refs_tok],
            hyp_tok,
            weights=(1/2, 1/2),
            smoothing_function=smooth
        )
        scores.append(score)
    max_score = max(scores)
    print(f"Example {i+1:2d} BLEU-2: {max_score*100:.2f}")

Example  1 BLEU-2: 38.70
Example  2 BLEU-2: 20.41
Example  3 BLEU-2: 23.57
Example  4 BLEU-2: 10.38
Example  5 BLEU-2: 23.57
Example  6 BLEU-2: 16.89
Example  7 BLEU-2: 9.69
Example  8 BLEU-2: 20.41
Example  9 BLEU-2: 25.68
Example 10 BLEU-2: 14.63
Example 11 BLEU-2: 21.38
Example 12 BLEU-2: 20.41
Example 13 BLEU-2: 20.41
Example 14 BLEU-2: 21.08
Example 15 BLEU-2: 36.55
Example 16 BLEU-2: 20.41
Example 17 BLEU-2: 20.41
Example 18 BLEU-2: 29.84
Example 19 BLEU-2: 35.83
Example 20 BLEU-2: 35.83
Example 21 BLEU-2: 24.76
Example 22 BLEU-2: 30.61
Example 23 BLEU-2: 33.83
Example 24 BLEU-2: 20.41
Example 25 BLEU-2: 24.69
Example 26 BLEU-2: 33.83
Example 27 BLEU-2: 15.32
Example 28 BLEU-2: 18.26
Example 29 BLEU-2: 49.93
Example 30 BLEU-2: 18.26
Example 31 BLEU-2: 21.08
Example 32 BLEU-2: 21.10
Example 33 BLEU-2: 13.09
Example 34 BLEU-2: 18.87
Example 35 BLEU-2: 3.38
Example 36 BLEU-2: 40.84
Example 37 BLEU-2: 25.14
Example 38 BLEU-2: 15.19
Example 39 BLEU-2: 20.41
Example 40 BLEU-2: 21.08
Ex

In [ ]:
# Corpus-level BLEU-2
# corpus_bleu expects list-of-list-of-tokens refs, and list-of-tokens hyps
corpus_score = corpus_bleu(
    tokenized_refs,
    tokenized_hyps,
    weights=(1/2, 1/2),
    smoothing_function=smooth
)
print(f"\nCorpus BLEU-2: {corpus_score*100:.2f}")


Corpus BLEU-2: 46.94


In [ ]:
# Sentence-level BLEU-3
smooth = SmoothingFunction().method1
for i, (refs_per_sample, hyp_tok) in enumerate(zip(tokenized_refs, tokenized_hyps)):
    scores = []
    for refs_tok in refs_per_sample:
        score = sentence_bleu(
            [refs_tok],
            hyp_tok,
            weights=(1/3, 1/3, 1/3),
            smoothing_function=smooth
        )
        scores.append(score)
    max_score = max(scores)
    print(f"Example {i+1:2d} BLEU-3: {max_score*100:.2f}")

Example  1 BLEU-3: 20.12
Example  2 BLEU-3: 8.41
Example  3 BLEU-3: 9.26
Example  4 BLEU-3: 3.55
Example  5 BLEU-3: 9.26
Example  6 BLEU-3: 6.63
Example  7 BLEU-3: 3.81
Example  8 BLEU-3: 8.41
Example  9 BLEU-3: 16.69
Example 10 BLEU-3: 6.03
Example 11 BLEU-3: 8.41
Example 12 BLEU-3: 8.41
Example 13 BLEU-3: 8.41
Example 14 BLEU-3: 8.22
Example 15 BLEU-3: 24.81
Example 16 BLEU-3: 8.41
Example 17 BLEU-3: 8.41
Example 18 BLEU-3: 10.06
Example 19 BLEU-3: 10.95
Example 20 BLEU-3: 10.95
Example 21 BLEU-3: 16.69
Example 22 BLEU-3: 18.59
Example 23 BLEU-3: 25.89
Example 24 BLEU-3: 8.41
Example 25 BLEU-3: 8.27
Example 26 BLEU-3: 25.89
Example 27 BLEU-3: 6.03
Example 28 BLEU-3: 7.47
Example 29 BLEU-3: 30.55
Example 30 BLEU-3: 7.47
Example 31 BLEU-3: 8.22
Example 32 BLEU-3: 7.99
Example 33 BLEU-3: 5.39
Example 34 BLEU-3: 7.41
Example 35 BLEU-3: 2.42
Example 36 BLEU-3: 27.73
Example 37 BLEU-3: 8.37
Example 38 BLEU-3: 9.87
Example 39 BLEU-3: 8.41
Example 40 BLEU-3: 8.22
Example 41 BLEU-3: 10.95
Exa

In [ ]:
# Corpus-level BLEU-3
# corpus_bleu expects list-of-list-of-tokens refs, and list-of-tokens hyps
corpus_score = corpus_bleu(
    tokenized_refs,
    tokenized_hyps,
    weights=(1/3, 1/3, 1/3),
    smoothing_function=smooth
)
print(f"\nCorpus BLEU-3: {corpus_score*100:.2f}")


Corpus BLEU-3: 35.25


In [ ]:
# Sentence-level BLEU-4
smooth = SmoothingFunction().method1
for i, (refs_per_sample, hyp_tok) in enumerate(zip(tokenized_refs, tokenized_hyps)):
    scores = []
    for refs_tok in refs_per_sample:
        score = sentence_bleu(
            [refs_tok],
            hyp_tok,
            weights=(1/4, 1/4, 1/4, 1/4),
            smoothing_function=smooth
        )
        scores.append(score)
    max_score = max(scores)
    print(f"Example {i+1:2d} BLEU-4: {max_score*100:.2f}")

Example  1 BLEU-4: 10.21
Example  2 BLEU-4: 5.61
Example  3 BLEU-4: 6.03
Example  4 BLEU-4: 2.10
Example  5 BLEU-4: 6.03
Example  6 BLEU-4: 4.32
Example  7 BLEU-4: 2.48
Example  8 BLEU-4: 5.61
Example  9 BLEU-4: 7.79
Example 10 BLEU-4: 4.02
Example 11 BLEU-4: 5.61
Example 12 BLEU-4: 5.61
Example 13 BLEU-4: 5.61
Example 14 BLEU-4: 5.31
Example 15 BLEU-4: 11.95
Example 16 BLEU-4: 5.61
Example 17 BLEU-4: 5.61
Example 18 BLEU-4: 6.07
Example 19 BLEU-4: 6.29
Example 20 BLEU-4: 6.29
Example 21 BLEU-4: 7.97
Example 22 BLEU-4: 8.43
Example 23 BLEU-4: 19.69
Example 24 BLEU-4: 5.61
Example 25 BLEU-4: 4.94
Example 26 BLEU-4: 19.69
Example 27 BLEU-4: 4.02
Example 28 BLEU-4: 4.94
Example 29 BLEU-4: 13.97
Example 30 BLEU-4: 4.94
Example 31 BLEU-4: 5.31
Example 32 BLEU-4: 5.11
Example 33 BLEU-4: 3.60
Example 34 BLEU-4: 4.83
Example 35 BLEU-4: 2.15
Example 36 BLEU-4: 13.35
Example 37 BLEU-4: 4.97
Example 38 BLEU-4: 4.52
Example 39 BLEU-4: 5.61
Example 40 BLEU-4: 5.31
Example 41 BLEU-4: 6.29
Example 42

In [ ]:
# Corpus-level BLEU-4
# corpus_bleu expects list-of-list-of-tokens refs, and list-of-tokens hyps
corpus_score = corpus_bleu(
    tokenized_refs,
    tokenized_hyps,
    weights=(1/4, 1/4, 1/4, 1/4),
    smoothing_function=smooth
)
print(f"\nCorpus BLEU-4: {corpus_score*100:.2f}")


Corpus BLEU-4: 28.16


In [ ]:
# Go on to calculate ROUGE scores
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=76d23b64cabd404df8a95d97c87a4f671b86bc90731d1bfea2da7f7f0cf63bf2
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import nltk
from collections import Counter

# Ensure tokenizer
nltk.download('punkt', quiet=True)

def rouge_n(ref: str, hyp: str, n: int = 4):
    ref_toks = nltk.word_tokenize(ref.lower())
    hyp_toks = nltk.word_tokenize(hyp.lower())
    ref_ngrams = list(nltk.ngrams(ref_toks, n))
    hyp_ngrams = list(nltk.ngrams(hyp_toks, n))
    ref_counts = Counter(ref_ngrams)
    hyp_counts = Counter(hyp_ngrams)
    overlap = sum(min(ref_counts[ng], hyp_counts[ng]) for ng in ref_counts)
    recall = overlap / max(len(ref_ngrams), 1)
    precision = overlap / max(len(hyp_ngrams), 1)
    f1 = 2 * recall * precision / (recall + precision + 1e-8)
    return (recall, precision, f1)

# Compute ROUGE-2
all_recalls, all_precisions, all_f1s = [], [], []
for refs, pred in zip(all_references, predictions):
    recalls_per_sample, precisions_per_sample, f1s_per_sample = [], [], []
    for ref in refs:
        r, p, f = rouge_n(ref, pred, n=2)
        recalls_per_sample.append(r)
        precisions_per_sample.append(p)
        f1s_per_sample.append(f)

    max_score = max(f1s_per_sample)
    max_index = f1s_per_sample.index(max_score)
    all_recalls.append(recalls_per_sample[max_index])
    all_precisions.append(precisions_per_sample[max_index])
    all_f1s.append(f1s_per_sample[max_index])
    print(f"REF:  {refs[max_index]!r}")
    print(f"HYP:  {pred!r}")
    print(f"   ROUGE-2 Recall:    {recalls_per_sample[max_index] * 100:.2f}%")
    print(f"   ROUGE-2 Precision: {precisions_per_sample[max_index] * 100:.2f}%")
    print(f"   ROUGE-2 F1:        {f1s_per_sample[max_index] * 100:.2f}%\n")

Streaming output truncated to the last 5000 lines.
   ROUGE-2 F1:        66.67%

REF:  'There are several vehicles on the road at the roundabout, And many cars are parked on both sides of the road next to the roundabout .'
HYP:  'bout is on the grass next to some trees .'
   ROUGE-2 Recall:    7.69%
   ROUGE-2 Precision: 22.22%
   ROUGE-2 F1:        11.43%

REF:  'The roundabout is on the grass next to trees and buildings .'
HYP:  'bout is next to some trees and grass .'
   ROUGE-2 Recall:    18.18%
   ROUGE-2 Precision: 25.00%
   ROUGE-2 F1:        21.05%

REF:  'The roundabout is on the bare land next to buildings and trees .'
HYP:  'bout is on the grass next to some trees .'
   ROUGE-2 Recall:    33.33%
   ROUGE-2 Precision: 44.44%
   ROUGE-2 F1:        38.10%

REF:  'The roundabout is on the grass next to trees and buildings .'
HYP:  'y is next to some trees and buildings .'
   ROUGE-2 Recall:    36.36%
   ROUGE-2 Precision: 50.00%
   ROUGE-2 F1:        42.11%

REF:  'The roundabou

In [ ]:
# Report overall averages
avg_r = sum(all_recalls) / len(all_recalls)
avg_p = sum(all_precisions) / len(all_precisions)
avg_f = sum(all_f1s) / len(all_f1s)
print("=== AVERAGE ROUGE-2 METRICS ===")
print(f"Recall:    {avg_r*100:.2f}")
print(f"Precision: {avg_p*100:.2f}")
print(f"F1:        {avg_f*100:.2f}")

=== AVERAGE ROUGE-2 METRICS ===
Recall:    23.28
Precision: 28.88
F1:        25.38


In [ ]:
# Compute ROUGE-3
all_recalls, all_precisions, all_f1s = [], [], []
for refs, pred in zip(all_references, predictions):
    recalls_per_sample, precisions_per_sample, f1s_per_sample = [], [], []
    for ref in refs:
        r, p, f = rouge_n(ref, pred, n=3)
        recalls_per_sample.append(r)
        precisions_per_sample.append(p)
        f1s_per_sample.append(f)

    max_score = max(f1s_per_sample)
    max_index = f1s_per_sample.index(max_score)
    all_recalls.append(recalls_per_sample[max_index])
    all_precisions.append(precisions_per_sample[max_index])
    all_f1s.append(f1s_per_sample[max_index])
    print(f"REF:  {refs[max_index]!r}")
    print(f"HYP:  {pred!r}")
    print(f"   ROUGE-3 Recall:    {recalls_per_sample[max_index] * 100:.2f}%")
    print(f"   ROUGE-3 Precision: {precisions_per_sample[max_index] * 100:.2f}%")
    print(f"   ROUGE-3 F1:        {f1s_per_sample[max_index] * 100:.2f}%\n")

Streaming output truncated to the last 5000 lines.
   ROUGE-3 F1:        50.00%

REF:  'The roundabout connects three roads and a sculpture is in the middle of the roundabout .'
HYP:  'bout is on the grass next to some trees .'
   ROUGE-3 Recall:    0.00%
   ROUGE-3 Precision: 0.00%
   ROUGE-3 F1:        0.00%

REF:  'There are several moving vehicles on the road at the roundabout, Many buildings and some trees and a few lawns around the roundabout .'
HYP:  'bout is next to some trees and grass .'
   ROUGE-3 Recall:    4.35%
   ROUGE-3 Precision: 14.29%
   ROUGE-3 F1:        6.67%

REF:  'The roundabout is on the bare land next to buildings and trees .'
HYP:  'bout is on the grass next to some trees .'
   ROUGE-3 Recall:    9.09%
   ROUGE-3 Precision: 12.50%
   ROUGE-3 F1:        10.53%

REF:  'The roundabout is on the grass next to trees and buildings .'
HYP:  'y is next to some trees and buildings .'
   ROUGE-3 Recall:    20.00%
   ROUGE-3 Precision: 28.57%
   ROUGE-3 F1:        23.5

In [ ]:
# Report overall averages
avg_r = sum(all_recalls) / len(all_recalls)
avg_p = sum(all_precisions) / len(all_precisions)
avg_f = sum(all_f1s) / len(all_f1s)
print("=== AVERAGE ROUGE-3 METRICS ===")
print(f"Recall:    {avg_r*100:.2f}")
print(f"Precision: {avg_p*100:.2f}")
print(f"F1:        {avg_f*100:.2f}")

=== AVERAGE ROUGE-3 METRICS ===
Recall:    15.11
Precision: 19.06
F1:        16.62


In [ ]:
# Compute ROUGE-4
all_recalls, all_precisions, all_f1s = [], [], []
for refs, pred in zip(all_references, predictions):
    recalls_per_sample, precisions_per_sample, f1s_per_sample = [], [], []
    for ref in refs:
        r, p, f = rouge_n(ref, pred, n=4)
        recalls_per_sample.append(r)
        precisions_per_sample.append(p)
        f1s_per_sample.append(f)

    max_score = max(f1s_per_sample)
    max_index = f1s_per_sample.index(max_score)
    all_recalls.append(recalls_per_sample[max_index])
    all_precisions.append(precisions_per_sample[max_index])
    all_f1s.append(f1s_per_sample[max_index])
    print(f"REF:  {refs[max_index]!r}")
    print(f"HYP:  {pred!r}")
    print(f"   ROUGE-4 Recall:    {recalls_per_sample[max_index] * 100:.2f}%")
    print(f"   ROUGE-4 Precision: {precisions_per_sample[max_index] * 100:.2f}%")
    print(f"   ROUGE-4 F1:        {f1s_per_sample[max_index] * 100:.2f}%\n")

Streaming output truncated to the last 5000 lines.
   ROUGE-4 F1:        42.86%

REF:  'The roundabout connects three roads and a sculpture is in the middle of the roundabout .'
HYP:  'bout is on the grass next to some trees .'
   ROUGE-4 Recall:    0.00%
   ROUGE-4 Precision: 0.00%
   ROUGE-4 F1:        0.00%

REF:  'The roundabout with three exits and entrances is in the residential area .'
HYP:  'bout is next to some trees and grass .'
   ROUGE-4 Recall:    0.00%
   ROUGE-4 Precision: 0.00%
   ROUGE-4 F1:        0.00%

REF:  'The roundabout connects four roads and a building is next to the roundabout .'
HYP:  'bout is on the grass next to some trees .'
   ROUGE-4 Recall:    0.00%
   ROUGE-4 Precision: 0.00%
   ROUGE-4 F1:        0.00%

REF:  'The roundabout is on the grass next to trees and buildings .'
HYP:  'y is next to some trees and buildings .'
   ROUGE-4 Recall:    11.11%
   ROUGE-4 Precision: 16.67%
   ROUGE-4 F1:        13.33%

REF:  'The roundabout with four exits and entr

In [ ]:
# Report overall averages
avg_r = sum(all_recalls) / len(all_recalls)
avg_p = sum(all_precisions) / len(all_precisions)
avg_f = sum(all_f1s) / len(all_f1s)
print("=== AVERAGE ROUGE-4 METRICS ===")
print(f"Recall:    {avg_r*100:.2f}")
print(f"Precision: {avg_p*100:.2f}")
print(f"F1:        {avg_f*100:.2f}")

=== AVERAGE ROUGE-4 METRICS ===
Recall:    10.74
Precision: 13.93
F1:        11.96


In [ ]:
import os

save_dir = "/content/drive/MyDrive/DI725 - Transformers and Attention-based Deep Networks/Term Project/siglip-gpt2-custom_vlm_finetuned"
os.makedirs(save_dir, exist_ok=True)

# Save tokenizer
tokenizer.save_pretrained(save_dir)

# Save merged language model (now a clean GPT-2)
model.language_model.save_pretrained(save_dir)

# Save vision encoder and image processor
model.vision_model.save_pretrained(f"{save_dir}/vision_encoder")
image_processor.save_pretrained(f"{save_dir}/vision_encoder")

# Save vision projection layer
torch.save(model.vision_proj.state_dict(), f"{save_dir}/vision_proj.pt")

# Save config for reinitialization
import json
config = {
    "vision_encoder_path": "vision_encoder",
    "language_model_path": ".",
    "vision_proj_path": "vision_proj.pt",
    "vision_hidden_size": model.vision_proj.in_features,
    "language_hidden_size": model.vision_proj.out_features
}
with open(os.path.join(save_dir, "custom_vlm_config.json"), "w") as f:
    json.dump(config, f, indent=2)


In [ ]:
print("DONE")

DONE
